# Pipeline

In [1]:
import sys

sys.path.append("/.../pipeline_imaging/imaging")

In [2]:
import imaging_utils
import os

#  Folder structures

In [3]:
# Pooled date folders across three sites
# step1 (pooled)/
# └── Optomed/
#     ├── UAB_Optomed_20240701-20240731
#     ├── UCSD_Optomed_20240701-20240731
#     └── UW_Optomed_20240708-20240716
#         ├── AIREADI-1353_2.25.297362505978695678901027013029982164657.dcm
#         ├── AIREADI-1353_2.25.297362505978695678901027013029982164659.dcm
#         └── ... (additional DICOM files)

# => optomed.organize =>


# DICOM files orangnized by protocols 6 subfolders
# step2/
# └── Optomed/
#     ├── non_functional_dicom
#     ├── missing_critical_information
#     └── optomed_mac_or_disk_centered_cfp
#         ├── optomed_mac_or_disk_centered_cfp_4001_2.25.2149858204032225975324541441048371422686.dcm
#         ├── optomed_mac_or_disk_centered_cfp_7003_2.25.2261269346246836445793863579426017981374.dcm
#         └── ... (additional DICOM files)

#  => eidon_instance.convert =>

# DICOM files are formatted to be NEMA compliant, still organized by protocols
# step3/
# └─── Optomed/
#     └── optomed_mac_or_disk_centered_cfp
#         ├── converted_optomed_mac_or_disk_centered_cfp_7001_2.25.2178672832739095022877980387739000898349.dcm
#         ├── converted_optomed_mac_or_disk_centered_cfp_7003_2.25.2261269346246836445793863579426017981374.dcm
#         └── ... (additional DICOM files)


#  => imaging_utils.format_file => (this process is univeral to all images, not eidon specific)
#  => optomed_instance.metadata =>

# De-identified again, file renamed, and organized in a final structure for data relase
# step4/
#   └─── Optomed/
#      retinal_photography/
#      ├── imaging/
#      │   ├──cfp/
#      │   │   ├── optomed_aurora
#      │   │   │   ├── 1001
#      │   │   │   ├── 1002
#      │   │   │   ├── 1003
#      │   └── └── └── └── file1.dcm
#      │   └── └── └── └── file2.dcm


# step_metadata/
#   └─── Optomed/
#         ├── retinal_photography/
#              ├── filename1.json
#              ├── filename2.json
#              ├── filename3.json
#              └── ... (additional JSON files)

# Optomed instance

In [ ]:
from imaging_optomed_retinal_photography_root import Optomed

optomed_instance = Optomed()

In [11]:
device = "Optomed"

main = f"/Volumes"

step1 = f"{main}/{device}"
step2 = f"{main}/step2/{device}"
step3 = f"{main}/step3/{device}"
step4 = f"{main}/step4/{device}"
step_metadata = f"{main}/metadata"

# Organize Optomed files by protocol


In [6]:
folders = imaging_utils.list_subfolders(step1)

for batch_folder in folders:
    filtered_list = imaging_utils.get_filtered_file_names(batch_folder)

    for file in filtered_list:
        organize_result = optomed_instance.organize(file, step2)

# Convert DICOM files to NEMA compliant DICOM files

In [7]:
# These are fixed options for protocols
protocols = ["optomed_mac_or_disk_centered_cfp"]

for protocol in protocols:
    output = f"{step3}/{protocol}"
    if not os.path.exists(output):
        os.makedirs(output)

    files = imaging_utils.get_filtered_file_names(f"{step2}/{protocol}")

    for file in files:
        optomed_instance.convert(file, output)

# Files are de-identified again, file renamed, and organized in a final structure for data relase+ output metadata

In [13]:
for folder in [step3]:
    filelist = imaging_utils.get_filtered_file_names(folder)

    for file in filelist:
        full_file_path = imaging_utils.format_file(file, step4)
        optomed_instance.metadata(full_file_path, f"{step_metadata}/{device}")